In [2]:
import os

In [3]:
pwd

'/home/shazam/Kidney_Disease_Classification/research'

In [4]:
os.chdir("../")

In [5]:
pwd

'/home/shazam/Kidney_Disease_Classification'

In [17]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [18]:
from cnn_classifier.constants import *
from cnn_classifier.utils.common import read_yaml, create_directories

In [19]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [20]:
import os
import zipfile
import gdown
from cnn_classifier import logger
from cnn_classifier.utils.common import get_size

In [24]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        '''
            Fetch data from the url
        '''

        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs('artifacts/data_ingestion', exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id, zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        """
            zip_file_path: str
            Extracts the zip file into the data directory
            Function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

            

In [25]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-26 22:16:48,133: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-02-26 22:16:48,134: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-26 22:16:48,135: INFO: common: created directory at: artifacts]
[2024-02-26 22:16:48,135: INFO: common: created directory at: artifacts/data-ingestion]
[2024-02-26 22:16:48,136: INFO: 1975804308: Downloading data from https://drive.google.com/file/d/1f2YFj0UGUtmvLvo81d9nF6PSyd45WcpN/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1f2YFj0UGUtmvLvo81d9nF6PSyd45WcpN
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1f2YFj0UGUtmvLvo81d9nF6PSyd45WcpN&confirm=t&uuid=edeadd76-2106-4496-a4b9-e485c301c4c5
To: /home/shazam/Kidney_Disease_Classification/artifacts/data_ingestion/data.zip
100%|██████████| 1.63G/1.63G [04:38<00:00, 5.84MB/s]

[2024-02-26 22:21:28,342: INFO: 1975804308: Downloaded data from https://drive.google.com/file/d/1f2YFj0UGUtmvLvo81d9nF6PSyd45WcpN/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


OSError: [Errno 28] No space left on device